<a href="https://colab.research.google.com/github/brandon-birchall/AutoEncoder/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Frame interpolation

## Setting up

In [ ]:
!pip install tfds-nightly imageio
!pip install git+https://github.com/tensorflow/docs

     |████████████████████████████████| 4.0MB 4.0MB/s 
  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-h0lzzvph
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-h0lzzvph
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0964cc6614d3285dc9513b812a211556ff114c7fc_-cp37-none-any.whl size=154244 sha256=1024e98db0c548099a1a39d8a9f4309cf30aa01ff783a7a28ada85f745e9bb80
  Stored in directory: /tmp/pip-ephem-wheel-cache-vx8rvczv/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


In [ ]:
!rm out/*.gif

rm: cannot remove 'out/*.gif': No such file or directory


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time

import tensorflow_docs.vis.embed as embed

from IPython import display

In [ ]:
ds = tfds.load('davis', split='train', shuffle_files=True)

videos = []
for x in iter(ds):
  frames = tf.math.reduce_mean(x['video']['frames'][:, :480, :640, :3], axis=3)
  #frames = x['video']['frames'][:, :480, :640, :3]
  videos.append(tf.cast(frames, tf.float32) / 255)
videos = tf.concat(videos, 0)

Dataset davis downloaded and prepared to /root/tensorflow_datasets/davis/480p/2.1.0. Subsequent calls will reuse this data.


In [ ]:
videos = tf.reshape(videos, [4209, 480, 640, 1])

In [ ]:
def to_gif(images, filename):
  anim_file = 'out/' + filename + ".gif"
  images = images.numpy()
  with imageio.get_writer(anim_file, mode='I') as writer:
    for i in range(len(images)):
      writer.append_data(images[i])


  embed.embed_file(anim_file)

In [ ]:
#to_gif(videos[0], 'test')

# The autoencoder
First, I will train a simple convolutional autoencoder. This will take our input images and translate it into a lower-dimensional representation and later reconstruct the original image.

In our Frame interpolator, we will use these lower-dimensional representations as inputs

In [ ]:
class Autoencoder(tf.keras.Model):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
      tf.keras.layers.InputLayer([480,640,1]),

      tf.keras.layers.Conv2D(20, [3,3], [1,1], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2)),

      tf.keras.layers.Conv2D(20, [3,3], [1,1], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2)),

      tf.keras.layers.Conv2D(20, [3,3], [1,1], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.MaxPooling2D(padding='same', pool_size=(2, 2)),

      tf.keras.layers.Conv2D(1, [3,3], [1,1], activation='relu', kernel_initializer='he_normal', padding='same'),

      tf.keras.layers.Dense(64),
      tf.keras.layers.Dense(64),
      tf.keras.layers.Dense(1),

      tf.keras.layers.Reshape([60, 80, 1])
     ])
    self.decoder = tf.keras.Sequential([
      tf.keras.layers.InputLayer([60,80,1]),

      tf.keras.layers.Dense(64),
      tf.keras.layers.Dense(64),
      tf.keras.layers.Dense(1),

      tf.keras.layers.Conv2DTranspose(1, [3,3], [1,1], activation='relu', kernel_initializer='he_normal', padding='same'),

      tf.keras.layers.Conv2DTranspose(20, [3,3], [2,2], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.Reshape((120, 160,20)),

      tf.keras.layers.Conv2DTranspose(20, [3,3], [2,2], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.Reshape([240,320,20]),

      tf.keras.layers.Conv2DTranspose(20, [3,3], [2,2], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.Reshape([480,640,20]),

      tf.keras.layers.Conv2D(1, [3, 3], activation='sigmoid', padding='same'),
      tf.keras.layers.Reshape([480,640,1]),

      tf.keras.layers.Dense(32),
      tf.keras.layers.Dropout(0.3),      
      tf.keras.layers.Dense(1),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

  def decode(self, image):
    return self.decoder(image)
  def encode(self, image):
    return self.encoder(image)

In [ ]:
print(auto_encoder.summary())

Model: "autoencoder_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 60, 80, 1)         7621      
_________________________________________________________________
sequential_3 (Sequential)    (None, 480, 640, 1)       7631      
Total params: 15,252
Trainable params: 15,252
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
auto_encoder = Autoencoder()
auto_encoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
auto_encoder.fit(videos, videos, epochs=3, batch_size = 32)

Epoch 1/3


KeyboardInterrupt: ignored

In [ ]:
encoded = auto_encoder.encode(tf.reshape(videos[:20], [20,480,640,1]))
demo = tf.cast(encoded * 255, tf.uint8)
plt.subplot(2,2,1)
plt.imshow(demo[0,:,:,0], cmap='gray')
plt.subplot(2,2,2)
plt.imshow(demo[0,:,:,1], cmap='gray')
plt.subplot(2,2,3)
plt.imshow(demo[0,:,:,2], cmap='gray')
plt.subplot(2,2,4)
plt.imshow(demo[0,:,:,3], cmap='gray')

to_gif(demo[:,:,:,0], 'interm')

In [ ]:
input = tf.cast(videos[:30] * 255, tf.uint8)
out = tf.cast(auto_encoder(videos[:30]) * 255, tf.uint8)
#print(out)
to_gif(input, 'input')
to_gif(out, 'result')

In [ ]:
interpolator = tf.keras.Sequential([
      tf.keras.layers.InputLayer([2,60,80,4]),
      tf.keras.layers.Reshape((60, 80, 8)),
      tf.keras.layers.Dense(64),
      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Dense(64),
      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Conv2D(4, [3,3], [1,1], activation='relu', kernel_initializer='he_normal', padding='same'),
      tf.keras.layers.Reshape([60,80,4]),
])